In [58]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import io
import cv2
import textwrap
import matplotlib.patches as patches
import math
import os
from pathlib import Path
from datetime import datetime
import json
import glob
import sys

seed = 42
np.random.seed(seed)

In [2]:
img_dir_path = Path("output", "images")
lbl_dir_path = Path("output", "labels")
today = f'{(int(datetime.now().strftime("%m%d")) - int("1001")):04}'
imgfn_prefix = f"image_{today}_"
lblfn_prefix = f"label_{today}"
startno = 0
endno = 1e8
wiki_maxnum = 100


In [3]:
def print_img(img, cvf = False):
    if cvf:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        # img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    plt.figure(figsize=(10, 10))#figsize=(5,5))
    plt.imshow(img)

In [59]:
def detect_strarea(image_name):
    moji = []
    string = []
    num = 1
    j = -1
    high_line = 10000

    tools = pyocr.get_available_tools()
    if len(tools) == 0:
        print("No OCR tool found")
        sys.exit(1)

    tool = tools[0]
    imgname = image_name
    img = cv2.imread(imgname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mser = cv2.MSER_create()
    mser.setMinArea(100)
    mser.setMaxArea(800)

    coordinates = mser.detectRegions(gray, None)

    for coord in coordinates:
        bbox = cv2.boundingRect(coord)
        x,y,w,h = bbox
        aspect_ratio = w / h
        if w < 10 or h < 10 or w/h > 5:
            continue
        moji.append([x, y, h, w])

    moji_sort = sorted(moji, key=lambda x: x[1])
    for i in range(len(moji_sort)):
        if high_line >= moji_sort[i][1]-10 and high_line <= (moji_sort[i][1] + moji_sort[i][3]+10):
            string[j].append(moji_sort[i])
        else:
            high_line = moji_sort[i][1]+moji_sort[i][3]/2
            string.append([])
            j = j + 1
            string[j].append(moji_sort[i])

    for i in range(len(string)):
        if len(string[i]) < 30:
            continue
        string_sort = sorted(string[i], key=lambda x: x[0])

        string_sort = np.array(string_sort)

        im = img[string_sort.min(axis=0)[1]-2:string_sort.min(axis=0)[1]+(string_sort.min(axis=0)[1]+string_sort.max(axis=0)[3]-string_sort.min(axis=0)[1]), string_sort.min(axis=0)[0]-2:string_sort.min(axis=0)[0]+(string_sort.max(axis=0)[0]-string_sort.min(axis=0)[0]+string_sort.max(axis=0)[2])]
    
        cv2.imwrite("result"+str(num)+".png", im)
    
        txt = tool.image_to_string(Image.open('result'+str(num)+'.png'), lang="eng", builder=pyocr.builders.TextBuilder(tesseract_layout=6))
        print(txt)

    num = num + 1
    cv2.imwrite("result"+str(num)+".png", img)

In [ ]:
def checkImg(file_path):
    pass

def trans(pts1):
    res = []
    w, h = 640, 480
    for p in pts1:
        newp = [p[0], h-p[1]]
        res.append(newp)
    return np.float32(res)

def get_filenames(img_dir_path):
    res = glob.glob(str(img_dir_path / f"{imgfn_prefix}*.png"))
    return res

def read_json(json_file_path):  
    with open(json_file_path, 'r') as f:  
        data = json.load(f)  
    return data 

def calculate_distance(point1, point2):  
    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2) 

def morf(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    res = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=2)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    res = cv2.morphologyEx(res, cv2.MORPH_OPEN, kernel, iterations=2)
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # res = cv2.morphologyEx(dst, cv2.MORPH_CLOSE, kernel, iterations=2)
    return res

def main():
    filenames = get_filenames(img_dir_path)
    jsondata = read_json(str(lbl_dir_path / f"{lblfn_prefix}.json"))
    width, height = 640, 480
    json_pt = 0
    for i in range(10):
        checkFlag = (i >= startno and i < endno)
        if not checkFlag:
            continue
        if not jsondata[json_pt]["image_name"] in filenames[i]:
            continue
        
        img =  cv2.imread(filenames[i])
        img = cv2.resize(img, (width, height))
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        denoised_image = cv2.bilateralFilter(gray_image, 9, 75, 75)
        im_edges = cv2.Canny(denoised_image, 20, 10, L2gradient=True)
        mimg = morf(im_edges)
        #_, binary_image = cv2.threshold(denoised_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        print_img(img, True)
        print_img(mimg, True)

        jd = jsondata[json_pt]
        print(jd)
        json_pt += 1

    # cv2.destroyAllWindows() 

main()
